<h2>Pre-processing</h2>
<h4>Dieser Code muss mit jeder Datei einzeln durchgeführt werden. </h4>

In [1]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
#from nltk.tokenize import sent_tokenize, word_tokenize

data = pd.read_csv("C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\train-neg.csv", header=None, engine='python')

data.head()

,0
0,Came in a brown half liter bottle with a good ...
1,"12 oz bottle poured into a pint glass, got a b..."
2,Appearance: Pours a rich amber-red color with ...
3,I can't imagine a worse winter warmer. Surely ...
4,"12 oz bottle best by 8/9/11, poured into a pin..."


In [2]:
len(data.shape)
data.shape[0]#-> 1500 Anzahl rows
len(data.columns) #-> 1 Anzahl columns
data.shape[1] #-> 1 Anzahl columns
data.loc[5,0]

"12 oz brown bottle.\t\tFoamy white head with not much retention. Plenty of noticeable carbonation. Virtually no lacing. Extremely watery; the faintest shade of dull straw coloration. Other than that, this beer is clear.\t\tCorn, rice, and a metallic graininess. Skunky aroma as well.\t\tA moderate wheat flavor joined by the usual rice and corn dynamics typical of a light lager. A very weak malt character and a touch of unflattering hops. The skunkiness returns and lasts throughout the aftertaste.\t\tVery light bodied; fizzy and grainy on the palate.\t\tThis is just an insult to the word 'beer' in terms of both the 3.5% abv and the flavor. I'd rather be drinking water. It's a shame that marketing is all it takes for a company to get rich off of such crap."

<h3>Entfernen der Satzzeichen</h3>

In [3]:
punctuation = [',', '.', ':', ';', '/', '\\', '?','!', '-', '"', '(', ')']
tempString = ''
stringSlice = ''

for x in range(0,data.shape[0],1): #replace funktioniert nicht?!
    #print('in row' + str(x))
    tempString = ''
    if not type(data.loc[x,0]) is float:
        for i in range(len(data.loc[x,0])):
            if (data.loc[x,0])[i] not in punctuation:
                if not ((data.loc[x,0])[i:i+1] == '\t'):
                    if i == 0:
                        tempString = (data.loc[x,0])[i]
                    else:
                        tempString += (data.loc[x,0])[i]
                else: # Falls ein Tab gefunden wurde wird dafür eine Leerstelle eingefügt
                    if i != 0:
                        tempString += ' '             
    data.loc[x,0] = tempString
data.head()

,0
0,Came in a brown half liter bottle with a good ...
1,12 oz bottle poured into a pint glass got a bo...
2,Appearance Pours a rich amberred color with a ...
3,I can't imagine a worse winter warmer Surely t...
4,12 oz bottle best by 8911 poured into a pint g...


In [4]:
tempString = ''


for x in range(0,data.shape[0],1): #finden doppelter Leerstellen
    #print('in row' + str(x))
    tempString = ''
    if not type(data.loc[x,0]) is float:
        for i in range(len(data.loc[x,0])):
            if not ((data.loc[x,0])[i:i+2] == '  '):
                    if i == 0:
                        tempString = (data.loc[x,0])[i]
                    else:
                        tempString += (data.loc[x,0])[i]
           #else:
                #print('Found!!!')
    data.loc[x,0] = tempString

data.head()

,0
0,Came in a brown half liter bottle with a good ...
1,12 oz bottle poured into a pint glass got a bo...
2,Appearance Pours a rich amberred color with a ...
3,I can't imagine a worse winter warmer Surely t...
4,12 oz bottle best by 8911 poured into a pint g...


In [5]:
data.to_csv('C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\PreprocessedStage1.csv')

<h2>Stopwords</h2>

In [8]:
import io 
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english')) 

counter = 0
stopwordsData = ''

for x in range(0,data.shape[0],1):
    #print('in row' + str(x))
    stopwordsData = ''
    counter = 0
    if not type(data.loc[x,0]) is float:
        for word in data.loc[x,0].split():
            #print('alive and in if loop ' + str(x))
            if counter == 0:
                #print(word)
                if word in stop_words:
                    #print('found stopword' + word)
                    #(data.loc[x,0]).replace(word, '')
                    counter = 1
                else:
                    counter = 1
                    stopwordsData = word
            else:
            #print('alive and in if loop ' + str(x))
                if word in stop_words:
                    print('found stopword ' + word)
                    #(data.loc[x,0]).replace(word, '')
                else:
                    stopwordsData += ' ' + word
    data.loc[x,0] = stopwordsData
data.head()

,0
0,Came brown half liter bottle good looking mult...
1,12 oz bottle poured pint glass got bottle holi...
2,Appearance Pours rich amberred color ton hop h...
3,I can't imagine worse winter warmer Surely wor...
4,12 oz bottle best 8911 poured pint glass A pen...


<h2>Wordnet</h2>

In [9]:
from nltk.corpus import wordnet

counter = 0

for x in range(0,data.shape[0],1): 
    #print('in row' + str(x))
    synString = ''
    if not type(data.loc[x,0]) is float:
        for word in data.loc[x,0].split():
            #print(word)
            syns = wordnet.synsets(word)
            if syns: #check if synset exists
                if counter == 0:
                    synString =(syns)[0].lemmas()[0].name()
                    #print(syns)
                    counter = 1
                else:
                    synString +=' ' + (syns)[0].lemmas()[0].name()
                    #print(syns)
                #hyper = ((syns)[0].hypernyms())# check if hypernyms exists
                #if hyper:
                #    if counter == 0:
                #        synString =((syns)[0].hypernyms())[0].lemmas()[0].name()
                #        print(syns)
                #        counter = 1
                #    else:
                #        synString +=' ' + ((syns)[0].hypernyms())[0].lemmas()[0].name()
                #        print(syns)
                        
            else:
                if counter == 0:
                    synString = word
                    counter = 1
                else:
                    synString += ' ' + word
                
    data.loc[x,0] = synString
#wordnet.synsets(word)

data.head()


,0
0,come brown one-half liter bottle good look mot...
1,twelve oz bottle pour pint glass get bottle v...
2,appearance pour rich_people amberred color sh...
3,iodine can't imagine worse winter heater sure...
4,twelve oz bottle best 8911 pour pint glass an...


<h2>Stemmer</h2>

In [10]:
counter = 0
stemmedData = ''

stemmer = SnowballStemmer("english")
for x in range(0,data.shape[0],1): #+1 bei shape  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!+1
    #print('in row' + str(x))
    stemmedData = ''
    if not type(data.loc[x,0]) is float:
        for word in data.loc[x,0].split():
            #print('alive and in if loop ' + str(x))
            if counter == 0:
                stemmedData = stemmer.stem(word)
                counter = 1
            else:
                stemmedData += ' ' + stemmer.stem(word)
    data.loc[x,0] = stemmedData

data.head()

,0
0,come brown one-half liter bottl good look motl...
1,twelv oz bottl pour pint glass get bottl vaca...
2,appear pour rich_peopl amber color short_ton ...
3,iodin can't imagin wors winter heater sure wo...
4,twelv oz bottl best 8911 pour pint glass angs...


<h2>Save data</h2>

In [11]:
preprocessedData = pd.DataFrame({'Text': data[0], 'Label': 0})
preprocessedData.head()

,Text,Label
0,come brown one-half liter bottl good look motl...,0
1,twelv oz bottl pour pint glass get bottl vaca...,0
2,appear pour rich_peopl amber color short_ton ...,0
3,iodin can't imagin wors winter heater sure wo...,0
4,twelv oz bottl best 8911 pour pint glass angs...,0


In [12]:
preprocessedData.to_csv('C:\\Users\\ojkbe\\OneDrive\\Documents\\7. Semester\\KI\\Übung\\preprocessedDataNeg.csv')